<a href="https://colab.research.google.com/github/rubenapf/AI-for-Developers/blob/main/202509_practical_ai_development_w4_d3_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG - v6

- Multiquery + Rerank

In [ ]:
#@title Setup better response formatting (adds line wrap)
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Install Dependencies

In [ ]:
%pip install "langchain==0.3.27" -qqq
%pip install "langchain-community==0.3.31" -qqq
%pip install "langchain-openai==0.3.35" -qqq
%pip install "langchain-chroma==0.2.6" -qqq
%pip install pypdf -qqq
%pip install gradio -qqq

#--------------------------------------------------------------------------------
# NEW IN v6: Install FlashRank
#--------------------------------------------------------------------------------
%pip install flashrank -qqq

## Configuration

In [ ]:
import os
from google.colab import userdata

# OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["CHROMA_API_KEY"] = userdata.get("CHROMA_API_KEY")
os.environ["CHROMA_TENANT"] = userdata.get("CHROMA_TENANT")

# Langsmith Environment Variables
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Code For All - LangChain Intro"

# Version Management
VERSION = "v6"

# Vector Database collection name
COLLECTION_NAME = f"bitcoin_docs_{VERSION}"


## Global Variables

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings Model
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# LLM
llm = ChatOpenAI(
    model="gpt-4o-mini"
)

# Classification LLM
classification_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0  # Deterministic for consistent classification
)

# Vector Database
vectorstore = Chroma(
  embedding_function=embeddings,
  collection_name=COLLECTION_NAME,  # Version-based naming
  chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
  tenant=os.getenv("CHROMA_TENANT"),
  database="code_for_all_rag_1"
)

# Parent splitter (large chunks for context)
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=240, # 20% of chunk_size
)

# Child splitter (small chunks for search)
# will generate aprox. 4 child documents per parent
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=60, # 15% of chunk_size
)

# InMemoryByteStore for parent documents
parent_docstore = InMemoryStore()

# Langchain Retriever
# - Component whose only job is to fetch relevant documents given a query.
# - It's like a “search engine” in your RAG pipeline.

# ParentDocumentRetriever
# - Stores small chunks in Chroma Cloud (for search)
# - Stores large parents in e.g. Google Drive (for better context)
parent_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=parent_docstore,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={"k": 3}
)

## Ingestion

In [ ]:
#@title Clean Text Function

import re

def clean_text(text: str) -> str:
    """
    TEXT PREPROCESSING - Same as v4
    """
    # Remove multiple whitespaces -> \s+ matches one or more whitespace characters (spaces, tabs, newlines) and replaces them with a single space " "
    text = re.sub(r'\s+', ' ', text)

    # Remove standalone page numbers -> remove a trailing number only if it's the final token
    text = re.sub(r'(?<=\.)\s*\d+\s*$', '', text)

    # Removes whitespace from both the beginning and the end of the string
    text = text.strip()

    return text

In [ ]:
#@title Topic Detection Function

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def detect_document_topic(documents) -> str:
    """
    DETECT DOCUMENT TOPIC - Same as v4
    """

    # Template for automatic topic detection
    topic_detection_template = ChatPromptTemplate.from_template(
      """
      Analyze the following document content and determine its primary topic.

      Document content:
      {content}

      Based on this content, what is the primary topic? Answer with a single word or short phrase (e.g., 'bitcoin', 'ethereum').

      Examples:
      If the document is about Bitcoin, answer: bitcoin
      If the document is about Ethereum, answer: ethereum
      If the document is about general crypto technology, answer: crypto

      Primary topic:
      """
    )

    topic_detection_chain = topic_detection_template | classification_llm | StrOutputParser()

    # Extract sample content from first few pages
    sample_content = ""
    for doc in documents[:3]:  # First 3 pages
        sample_content += doc.page_content + "\n\n"

    # Limit to 4000 characters to save costs
    sample_content = sample_content[:4000]

    # Use LLM to detect topic
    detected_topic = topic_detection_chain.invoke({
        "content": sample_content
    }).strip().lower()

    return detected_topic

In [ ]:
#@title Ingest function

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def ingest_documents(
    document_path: str,
    access_level: str = "public"
):
    """
    INGESTION PIPELINE - Same as v5

    Parent-Child Document Strategy
    1. Add documents with metadata
    2. ParentDocumentRetriever automatically:
       - Splits into parent chunks (1200 chars)
       - Splits parents into child chunks (400 chars)
       - Stores children in Chroma Cloud (searchable, persistent)
       - Stores parents in InMemoryByteStore (context, session-based)
       - Maps children → parents automatically
    3. Later retrieval automatically returns parent chunks

    Args:
        document_path (str): URL or local path to PDF
        access_level (str): "public" or "premium" (for access control demo)

    Returns:
        tuple: (num_chunks, detected_topic) - Number of chunks and detected topic
    """

    print("-" * 80)
    print(f"STARTING INGESTION PIPELINE - VERSION {VERSION}")
    print("-" * 80)

    #--------------------------------------------------------------------------------
    # STEP 1: LOAD DOCUMENTS
    #--------------------------------------------------------------------------------
    print("\n[1/5] Loading file from URL...")

    loader = PyPDFLoader(document_path)
    documents = loader.load()

    print(f"✓ Loaded {len(documents)} pages from file")

    #--------------------------------------------------------------------------------
    # STEP 2: AUTO-DETECT TOPIC
    #--------------------------------------------------------------------------------
    print(f"\n[2/5] Auto-detecting document topic...")

    detected_topic = detect_document_topic(documents)

    print(f"✓ Topic auto-detected: '{detected_topic}'")

    #--------------------------------------------------------------------------------
    # STEP 3: PREPROCESSING
    #--------------------------------------------------------------------------------
    print(f"\n[3/5] Applying text preprocessing...")

    for doc in documents:
      doc.page_content = clean_text(doc.page_content)

    print(f"✓ Cleaned {len(documents)} pages")

    #--------------------------------------------------------------------------------
    # STEP 4: ADD METADATA (CHANGED IN v5 - no chunks, still docs)
    #--------------------------------------------------------------------------------
    print(f"\n[4/5] Enriching chunks with metadata...")

    for doc in documents:
      # ADD new metadata (doesn't override existing)
      doc.metadata.update({
          'topic': detected_topic,
          'access_level': access_level,   # Used for access control
      })

    print(f"✓ Metadata enriched for all chunks:")

    #--------------------------------------------------------------------------------
    # STEP 5: USE ParentDocumentRetriever
    #--------------------------------------------------------------------------------
    print(f"\n[5/5] Processing with ParentDocumentRetriever...")

    # This one line does:
    # 1. Split each parent document into child chunks
    # 2. Embed and store child chunks in the vectorstore
    # 3. Store full parent documents in the docstore
    # 4. Link each child chunk to its parent via metadata["doc_id"]
    print(f"Adding {len(documents)} documents/pages (before split)")
    parent_retriever.add_documents(documents)

    keys = list(parent_docstore.yield_keys())
    print("Added", len(keys), "documents (after text split) to parent_docstore")
    for key, value in zip(keys, parent_docstore.mget(keys)):
        print("\n" + key, "=>", value)

    print("\n" + "="*80)
    print("✓ Parent chunks created and stored in InMemoryStore")
    print("✓ Child chunks embeddings created and stored in Chroma")

    print("\n" + "="*80)
    print("INGESTION COMPLETE")
    print("="*80)
    return len(documents), detected_topic




In [ ]:
#@title Database Reset (Helper)

# Clear InMemoryStore before ingestion
print("\nClearing InMemoryStore...")
keys = list(parent_docstore.yield_keys())
parent_docstore.mdelete(keys)
print("InMemoryStore reset! Current size:", len(list(parent_docstore.yield_keys())))

# Reset Chroma collection
print(f"\nResetting Chroma collection: {COLLECTION_NAME}...")
vectorstore.reset_collection()
print(f"✓ Collection '{COLLECTION_NAME}' reset!")

In [ ]:
#@title Execute Ingestion

ingest_documents("https://bitcoin.org/bitcoin.pdf")
#ingest_documents("/content/ethereum.pdf", "premium")

## Inference

In [ ]:
#@title Topic Classification Chain Creation

def create_classification_chain():
    """
    CREATE CLASSIFICATION CHAIN - NEW IN v4 - refactor and nehancement with conversation history

    Determines whether a user query is about Bitcoin, Ethereum, or both.
    This enables intelligent routing to relevant documents only.

    Returns:
        RunnableSequence: Classification chain
    """

    classification_template = ChatPromptTemplate.from_template(
      """
      Classify this question as about 'bitcoin', 'ethereum', or 'both'.
      Consider the conversation history for context (e.g., if user previously asked about Bitcoin, "it" might refer to Bitcoin).

      Conversation history:
      {history}

      Question: {query}

      Classification:
      """
    )

    classification_chain = classification_template | classification_llm | StrOutputParser()

    return classification_chain

In [ ]:
#@title RAG Chain Creation

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def create_rag_chain():
    """
    CREATE RAG CHAIN - NEW IN v4

    KEY DIFFERENCES FROM v3:
    - Includes conversation history in the prompt
    - Handles follow-up questions
    - More natural conversational flow

    Returns:
        RunnableSequence: RAG chain
    """

    rag_template = ChatPromptTemplate.from_template(
      """
      You are a helpful assistant answering questions about cryptocurrency whitepapers.

      Use the conversation history and provided documents to answer the current question.
      If the question references previous context (e.g., "it", "this", "that"), use the conversation history to understand what is being referenced.

      Instructions:
      - Answer clearly and concisely (max 2-3 paragraphs)
      - Use only the provided documents as context
      - If the question is a follow-up, maintain conversation continuity
      - If the answer is not in the documents, say "I don't have enough information to answer that question"

      Conversation History:
      {history}

      Documents:
      {context}

      Current Question: {query}

      Answer:
      """
    )

    rag_chain = rag_template | llm | StrOutputParser()

    return rag_chain

In [ ]:
#@title Full RAG Chain Creation

from langchain_core.runnables import RunnablePassthrough, RunnableParallel

def create_parent_child_rag_chain_integrated():
    """
    CREATE INTEGRATED PARENT-CHILD RAG CHAIN - NEW IN v5

    This chain combines:
    1. Formatting conversation history.
    2. Document retrieval using ParentDocumentRetriever.
    3. Formatting retrieved documents as context.
    4. Generating an answer using the RAG LLM chain.

    BENEFITS: One single trace in Langsmith that combines all steps.

    Returns:
        RunnableSequence: An integrated RAG chain that takes 'query' and 'chat_history'.
    """

    # The core RAG generation chain (template | LLM | parser) that expects context, query, and history
    rag_generation_chain = create_rag_chain()

    # Create the full chain that includes retrieval and context formatting
    # It takes 'query' and 'chat_history' as initial inputs
    full_rag_chain = (
        RunnableParallel({
            "context": (lambda x: x["query"]) | parent_retriever | (lambda docs: "\n\n".join([doc.page_content for doc in docs])),
            "query": RunnablePassthrough(),
            "history": (lambda x: format_chat_history(x["chat_history"], max_turns=5)),
        })
        | rag_generation_chain
    )
    return full_rag_chain

In [ ]:
#@title Format Chat History Helper

def format_chat_history(history: list, max_turns: int = 5) -> str:
    """
    FORMAT CHAT HISTORY - NEW IN v4

    Converts Gradio's messages format history into a readable string.

    WHY LIMIT TURNS?
    - Token limits: Long conversations exceed context window
    - Relevance: Recent turns are more relevant
    - Cost: Fewer tokens = lower cost
    - Performance: Faster processing

    GRADIO MESSAGES FORMAT:
    [
        {"role": "user", "content": "What is Bitcoin?"},
        {"role": "assistant", "content": "Bitcoin is..."},
        {"role": "user", "content": "How does it work?"},
        {"role": "assistant", "content": "It works by..."}
    ]

    Args:
        history (list): Gradio chat history in messages format
        max_turns (int): Maximum number of conversation turns to include

    Returns:
        str: Formatted conversation history
    """

    if not history:
        return "No previous conversation."

    # Limit to last N turns (each turn = user + assistant message)
    # Each turn consists of 2 messages (user + assistant)
    recent_history = history[-(max_turns * 2):]

    # Format as readable conversation
    formatted = []
    for message in recent_history:
        role = message["role"].capitalize()
        content = message["content"]
        formatted.append(f"{role}: {content}")

    return "\n".join(formatted)


In [ ]:
#@title Basic Inference

def inference(
    query: str,
    chat_history: list = None,
    user_access_level: str = "public"
) -> str:
    """
    INFERENCE PIPELINE - Same as previous versions

    Args:
        query (str): User's question
        chat_history (list): Gradio chat history
        user_access_level (str): "public" or "premium"

    Returns:
        str: Natural language answer
    """

    print("="*80)
    print(f"RUNNING INFERENCE - VERSION {VERSION}")
    print("="*80)

    #--------------------------------------------------------------------------------
    # STEP 1: FORMAT CONVERSATION HISTORY (NEW IN v4)
    #--------------------------------------------------------------------------------
    print(f"\n[1/6] Formatting conversation history...")

    formatted_history = format_chat_history(chat_history, max_turns=5)

    print(f"✓ History formatted ({len(chat_history or [])} messages)")

    #--------------------------------------------------------------------------------
    # STEP 2: TOPIC DETECTION (CHANGED IN v4 - refactor)
    #--------------------------------------------------------------------------------
    print(f"\n[2/6] Detecting document topic...")

    topic = create_classification_chain().invoke({
        "query": query,
        "history": formatted_history
    }).strip().lower()

    print(f"✓ Topic: '{topic}'")

    #--------------------------------------------------------------------------------
    # STEP 3: METADATA TOPIC FILTER - 2 alternatives: only topic or topic + access_control
    #--------------------------------------------------------------------------------
    print(f"\n[3/6] Building metadata filter...")

    filter_conditions = {}

    if topic in ['bitcoin', 'ethereum']:
        filter_conditions['topic'] = topic
        print(f"  ✓ Topic filter: {topic}")

    # conditions = []

    # if topic in ("bitcoin", "ethereum"):
    #     conditions.append({"topic": topic})
    #     print(f"  ✓ Topic filter: {topic}")
    # else:
    #     print(f"  ✓ Topic filter: none (searching both)")

    # if user_access_level == "public":
    #     conditions.append({"access_level": "public"})
    #     print(f"  ✓ Access filter: public only")
    # else:
    #     print(f"  ✓ Access filter: all content")

    # # Combine conditions properly for Chroma
    # filter_conditions = {"$and": conditions} if len(conditions) > 1 else (conditions[0] if conditions else None)

    # print(f"  Final filter: {filter_conditions or 'None'}")

    #--------------------------------------------------------------------------------
    # STEP 4: SIMILARITY SEARCH
    #--------------------------------------------------------------------------------
    print(f"\n[4/6] Performing similarity search...")

    if filter_conditions:
        results = vectorstore.similarity_search(query, k=3, filter=filter_conditions)
    else:
        results = vectorstore.similarity_search(query, k=3)

    #--------------------------------------------------------------------------------
    # STEP 5: FORMAT CONTEXT
    #--------------------------------------------------------------------------------

    print(f"\n[5/6] Formatting context for LLM...")

    context = "\n\n".join([doc.page_content for doc in results])

    #--------------------------------------------------------------------------------
    # STEP 6: GENERATE ANSWER BY INVOKING CHAIN (CHANGED IN v4 - refactor)
    #--------------------------------------------------------------------------------
    print(f"\n[6/6] Invoking RAG chain...")

    response = create_rag_chain().invoke({
        "context": context,  # Retrieved documents
        "query": query,       # User's question
        "history": formatted_history  # Conversation history
    })

    print("\n" + "="*80)
    print("INFERENCE COMPLETE")
    print("="*80)

    return response  # Returns string

### Advanced Retrieval Strategies

In [ ]:
#@title I - Parent-Child Retrieval

# =============================================================================
# INFERENCE RETRIEVAL STRATEGIES 1: PARENT-CHILD RETRIEVAL
# =============================================================================
def parent_children_inference(
    query: str,
    chat_history: list = None,
) -> str:
    """
    INFERENCE WITH PARENT-CHILD RETRIEVAL - Same as v5

    Features:
    - Uses ParentDocumentRetriever
    - Small chunks for search, large chunks for context
    """

    print("="*80)
    print(f"RUNNING INFERENCE (PARENT-CHILD) - VERSION {VERSION}")
    print("="*80)

    print(f"User query: {query}")
    print(f"Chat history: {chat_history}")
    print("="*80)

    #--------------------------------------------------------------------------------
    # STEP 1: FORMAT CONVERSATION HISTORY
    #--------------------------------------------------------------------------------
    print(f"\n[1/4] Formatting conversation history...")

    formatted_history = format_chat_history(chat_history, max_turns=5)

    print(f"✓ History formatted ({len(chat_history or [])} messages)")

    #--------------------------------------------------------------------------------
    # STEP 2: RETRIEVE WITH ParentDocumentRetriever
    #--------------------------------------------------------------------------------
    print(f"\n[2/4] Retrieving with ParentDocumentRetriever...")

    # Get parent documents
    results = parent_retriever.invoke(query)

    print(f"✓ Retrieved {len(results)} parent documents")
    print(results)

    #--------------------------------------------------------------------------------
    # STEP 3: FORMAT CONTEXT
    #--------------------------------------------------------------------------------

    print(f"\n[3/4] Formatting context for LLM...")

    context = "\n\n".join([doc.page_content for doc in results])

    #--------------------------------------------------------------------------------
    # STEP 4: GENERATE ANSWER BY INVOKING CHAIN
    #--------------------------------------------------------------------------------
    print(f"\n[4/4] Invoking RAG chain...")

    response = create_rag_chain().invoke({
        "context": context,  # Retrieved documents
        "query": query,       # User's question
        "history": formatted_history  # Conversation history
    })

    print("\n" + "="*80)
    print("INFERENCE WITH PARENT-CHILD RETRIEVAL COMPLETE")
    print("="*80)

    return response  # Returns string

In [ ]:
#@title I - Parent-Child Retrieval (One Chain)

# =============================================================================
# INFERENCE RETRIEVAL STRATEGIES 1: PARENT-CHILD RETRIEVAL
# =============================================================================
def parent_children_inference_one_chain(
    query: str,
    chat_history: list = None,
) -> str:
    """
    INFERENCE WITH PARENT-CHILD RETRIEVAL (ONE CHAIN) - Same as v5

    Features:
    - Uses ParentDocumentRetriever
    - Small chunks for search, large chunks for context
    """

    print("="*80)
    print(f"RUNNING INFERENCE FULL RAG CHAIN - PARENT-CHILD - VERSION {VERSION}")
    print("="*80)

    print(f"User query: {query}")
    print(f"Chat history: {chat_history}")
    print("="*80)

    print(f"\nInvoking Full RAG Chain with Parent-Child Retrieval...")

    response = create_parent_child_rag_chain_integrated().invoke({
        "query": query,       # User's question
        "chat_history": chat_history  # Pass the original chat_history (list) here
    })

    print("\n" + "="*80)
    print("INFERENCE WITH PARENT-CHILD RETRIEVAL (ONE CHAIN) COMPLETE")
    print("="*80)

    return response  # Returns string

In [ ]:
#@title II - Multi-query retrieval + Rerank

# =============================================================================
# INFERENCE RETRIEVAL STRATEGIES 2: MULTI-QUERY RETRIEVAL + RERANKING
# =============================================================================
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank

def inference_multiquery(
    query: str,
    chat_history: list = None,
) -> str:
    """
    INFERENCE WITH MULTI-QUERY RETRIEVAL - NEW in v6

    MULTI-QUERY:
    - LLM generates query variations
    - Each variation searches the child chunks
    - Combines and retrieves corresponding parent chunks

    RERANKING:
    - Cross-encoder scores each parent doc against ORIGINAL query
    - Reranks all parent documents by relevance
    - Keeps only top_n most relevant

    TRADE-OFFS:
    - Slower than single-strategy approaches (LLM calls for query expansion + reranking)
    - More expensive (more tokens processed)
    - BUT: Maximum quality for complex/ambiguous queries
    """

    print("="*80)
    print(f"RUNNING INFERENCE (MULTI-QUERY) - VERSION {VERSION}")
    print("="*80)

    print(f"User query: {query}")
    print(f"Chat history: {chat_history}")
    print("="*80)

    #--------------------------------------------------------------------------------
    # STEP 1: FORMAT CONVERSATION HISTORY
    #--------------------------------------------------------------------------------
    print(f"\n[1/4] Formatting conversation history...")

    formatted_history = format_chat_history(chat_history, max_turns=5)

    print(f"✓ History formatted ({len(chat_history or [])} messages)")

    #--------------------------------------------------------------------------------
    # STEP 2: RETRIEVE WITH MultiQueryRetriever - NEW in v6
    #--------------------------------------------------------------------------------
    print(f"\n[2/4] Performing multi-query retrieval...")

    """
    Creates a MultiQueryRetriever that:
    - Generates multiple query variations
    - Searches with each variation
    - Combines and deduplicates results
    """

    # 2.1 (TO COMPLETE)
    # Convert vectorstore to retriever
    # base_retriever =

    results = []

    print(f"\n✓ Base Retriever documents: {len(results)}")
    print(f"\n{results}")

    # 2.2 (TO COMPLETE)
    # Create a  multiquery retriever and pass the llm
    # multiquery_retriever =

    results = []

    print(f"\n✓ Multiquery Retriever documents: {len(results)}")
    print(f"\n{results}")

    # 2.3 (TO COMPLETE)
    # FlashrankRerank
    #
    # Cross-encoder is a transformer model: it takes a query and a document_i (encoded together)
    # and score how relevant each passage (document content) is to a query.
    #
    # Setup a compressor
    # compressor =


    # 2.4 (TO COMPLETE)
    # ContextualCompressionRetriever
    #
    # - Takes standard LangChain Document objects (with .page_content and .metadata).
    # - Handles the necessary conversion and adaptation so that external rankers
    #   or compressors (like FlashrankRerank, CohereRerank, etc.) can operate on them.
    # - Then maps the results back into LangChain Document objects.
    #
    # compression_retriever =


    compressed_docs = []

    print(f"\n✓ Reranked documents: {len(compressed_docs)}")
    print(f"\n{compressed_docs}")

    #--------------------------------------------------------------------------------
    # STEP 3: FORMAT CONTEXT
    #--------------------------------------------------------------------------------

    print(f"\n[3/4] Formatting context for LLM...")

    context = "\n\n".join([doc.page_content for doc in compressed_docs])

    #--------------------------------------------------------------------------------
    # STEP 4: GENERATE ANSWER BY INVOKING CHAIN
    #--------------------------------------------------------------------------------
    print(f"\n[4/4] Invoking RAG chain...")

    response = create_rag_chain().invoke({
        "context": context,  # Retrieved documents
        "query": query,       # User's question
        "history": formatted_history  # Conversation history
    })

    print("\n" + "="*80)
    print("INFERENCE WITH MULTI-QUERY RETRIEVAL COMPLETE")
    print("="*80)

    return response  # Returns string

In [ ]:
#@title Test Inference

#inference("What is Bitcoin?")

#parent_children_inference("What is Bitcoin?")

inference_multiquery("What is Bitcoin?")

## Gradio Demo

In [ ]:
def chat_inference(message, history, retrieval_strategy):
  """
  Gradio ChatInterface wrapper

  Args:
      message (str): Current user message
      history (list): Chat history
      retrieval_strategy (str): The selected retrieval strategy ("basic", "parent-child", "multi-query")

  Returns:
      str: Bot response
  """
  user_access_level = "premium"

  if retrieval_strategy == "multi-query":
    return inference_multiquery(
        query=message,
        chat_history=history
    )
  elif retrieval_strategy == "parent-child":
    return parent_children_inference(
        query=message,
        chat_history=history
    )
  else: # Default to 'basic' strategy
    return inference(
        query=message,
        chat_history=history,
        user_access_level=user_access_level
    )

In [ ]:
import gradio as gr

default_strategy = "multi-query"

demo = gr.ChatInterface(
    fn=chat_inference,
    type="messages",
    title=f"🪙 Bitcoin RAG Assistant ({VERSION})",
    description="Ask questions about crypto.",
    examples=[
        ["What is Bitcoin?", default_strategy],
        ["How does mining work?", default_strategy],
        ["What is proof of work?", default_strategy],
        ["Explain the blockchain structure", default_strategy],
        ["How are transactions verified?", default_strategy],
        ["What is the double-spending problem?", default_strategy]
    ],
    additional_inputs=[
        gr.Radio(
            ["basic", "parent-child", "multi-query"],
            label="Retrieval Strategy",
            value=default_strategy, # Default selected value
            interactive=True
        )
    ]
)

demo.launch(share=True, debug=True)